In [3]:
import sys
sys.path.append('../..')

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import feature_column as fc
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn import preprocessing

from utils.cuda_cluster import *
from utils.preprocessing import read_data, factorize_small_cardinality_with_index, df_to_tfdataset, split_join, get_media_index
from utils.evaluate import calculate_ctr, compute_rce, average_precision_score
import core.config as conf

/home/nyongja/anaconda3/envs/dask/lib/python3.7/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 40671 instead
  http_address["port"], self.http_server.port


In [4]:
client

Client Scheduler: tcp://127.0.0.1:41843 Dashboard: http://127.0.0.1:40671/status,Cluster Workers: 1 Cores: 1 Memory: 33.47 GB


## Load data

In [5]:
# data_path = conf.raw_data_path + '*' # for all dataset
data_path = conf.raw_data_path + 'part-00000'
ori_df = read_data(data_path)

In [6]:
ori_df

,text_ tokens,hashtags,tweet_id,present_media,present_links,present_domains,tweet_type,language,tweet_timestamp,engaged_with_user_id,engaged_with_user_follower_count,engaged_with_user_following_count,engaged_with_user_is_verified,engaged_with_user_account_creation,enaging_user_id,enaging_user_follower_count,enaging_user_following_count,enaging_user_is_verified,enaging_user_account_creation,engagee_follows_engager,reply_timestamp,retweet_timestamp,retweet_with_comment_timestamp,like_timestamp
npartitions=16,,,,,,,,,,,,,,,,,,,,,,,,
1,object,object,object,object,object,object,object,object,int32,object,int32,int32,bool,int32,object,int32,int32,bool,int32,bool,int32,int32,int32,int32
273328,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2983644,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3033902,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


- media target encoding(photo, video, gif)
- tweet_type
- language
- tweet_timestamp
- engaged_with_user_id
- engaged_with_user_follower_count
- engaging_user_id
- engaging_user_following_count
- engagee_follows_engager
- 'reply_timestamp', 'retweet_timestamp','retweet_with_comment_timestamp', 'like_timestamp'

In [7]:
df = ori_df[['present_media', 'tweet_type', #'language', 'tweet_timestamp', 
             'engaged_with_user_follower_count', 
             'enaging_user_following_count', 'engagee_follows_engager','reply_timestamp', 'retweet_timestamp','retweet_with_comment_timestamp', 'like_timestamp']]

## Preprocessing

In [8]:
df['is_reply'] = df['reply_timestamp'].compute().applymap(lambda x: 1 if x > 0 else 0).astype(np.int32)
df['is_retweet'] = df['retweet_timestamp'].compute().applymap(lambda x: 1 if x > 0 else 0).astype(np.int32)
df['is_comment'] = df['retweet_with_comment_timestamp'].compute().applymap(lambda x: 1 if x > 0 else 0).astype(np.int32)
df['is_like'] = df['like_timestamp'].compute().applymap(lambda x: 1 if x > 0 else 0).astype(np.int32)

In [9]:
df = df.drop('reply_timestamp', axis=1)
df = df.drop('retweet_timestamp', axis=1)
df = df.drop('retweet_with_comment_timestamp', axis=1)
df = df.drop('like_timestamp', axis=1)

In [10]:
df.head()

,present_media,tweet_type,engaged_with_user_follower_count,enaging_user_following_count,engagee_follows_engager,is_reply,is_retweet,is_comment,is_like
id,,,,,,,,,
1,<NA>,Retweet,2473,339,False,0,0,0,0
2,<NA>,TopLevel,4418640,1190,False,0,0,0,0
3,<NA>,TopLevel,219715,1473,False,0,0,0,1
4,<NA>,Retweet,2388283,200,False,0,0,0,0
5,Photo,TopLevel,414,379,True,0,0,0,1


### Media Target Encoding

In [11]:
df['present_media'] = df['present_media'].fillna( '' )

In [12]:
df,media_index, _ = factorize_small_cardinality_with_index(df, 'present_media', 'media_type')

In [14]:
media_index

,index,present_media
0,0,
1,1,GIF
2,2,GIF\tGIF
3,3,GIF\tGIF\tGIF
4,4,GIF\tPhoto
5,5,GIF\tVideo
6,6,Photo
7,7,Photo\tGIF
8,8,Photo\tPhoto
9,9,Photo\tPhoto\tPhoto


In [15]:
media_index = get_media_index(media_index)

AttributeError: 'str' object has no attribute 'str'

In [ ]:
media_index = media_index['number_of_media'].to_frame()

In [ ]:
media_index

In [ ]:
media_index = cudf.DataFrame(media_index)
media_index = dask_cudf.from_cudf(media_index, npartitions=2)

In [ ]:
df = df.merge(media_index, left_on="media_type", right_index = True, how='left')

In [ ]:
df = df.drop('present_media', axis = 1)
df = df.drop('media_type', axis = 1)

### Language Encoding

In [ ]:
#df, idx_to_language, _ = factorize_small_cardinality_with_index(df, 'language', 'language_encode') # how language encoding??

In [ ]:
#df = df.drop('language', axis = 1)

In [ ]:
#df.head()

### Tweet Type Encoding

In [ ]:
df, _= factorize_small_cardinality_with_index(df, 'tweet_type', 'tweet_type_encode') # how language encoding??

In [ ]:
df = df.drop('tweet_type', axis = 1)

In [ ]:
df.head()

In [ ]:
df = df.astype(np.int64)

In [ ]:
df.to_parquet(conf.data_root + "preprocessing/ffnn_prep.parquet")

## Modeling

### Like

In [24]:
df_y = df[['is_reply', 'is_retweet', 'is_comment', 'is_like']]
df_x = df.drop(['is_reply', 'is_retweet', 'is_comment', 'is_like'], axis = 1)

#### Scaling

In [26]:
standard_scaler = preprocessing.StandardScaler()
standard_scaler.fit(df_x.compute().as_matrix())

StandardScaler()

In [27]:
ss = standard_scaler.transform(df_x.compute().as_matrix())

In [28]:
df_x = pd.DataFrame(ss, columns = df_x.columns)

In [29]:
X_train = df_x
y_train_like = df_y['is_like'].to_frame()

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train_like.compute(), test_size=0.2)

In [31]:
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.2)

In [32]:
model = Sequential([
    Dense(16, activation = 'relu', input_dim = X_test.shape[1]),
    Dense(8, activation = 'relu'),
    Dense(4, activation = 'relu'),
    Dense(2, activation = 'softmax')
])

In [33]:
model.compile(
    optimizer = 'adam',
    loss = 'sparse_categorical_crossentropy', # softmax : sparse_categorical_crossentropy, sigmoid : binary_crossentropy
    metrics=['accuracy'])

In [34]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                96        
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 36        
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 10        
Total params: 278
Trainable params: 278
Non-trainable params: 0
_________________________________________________________________


In [35]:
model.fit(
    x = X_train,
    y = y_train.to_pandas(),
    validation_data=(X_val, y_val.to_pandas()),
    epochs=5,
    batch_size=64
)

Epoch 1/5
37924/37924 [==============================] - 13s 329us/step - loss: 0.6417 - accuracy: 0.6210 - val_loss: 0.6384 - val_accuracy: 0.6220
Epoch 2/5
37924/37924 [==============================] - 12s 329us/step - loss: 0.6388 - accuracy: 0.6230 - val_loss: 0.6378 - val_accuracy: 0.6230
Epoch 3/5
37924/37924 [==============================] - 12s 330us/step - loss: 0.6382 - accuracy: 0.6228 - val_loss: 0.6377 - val_accuracy: 0.6242
Epoch 4/5
37924/37924 [==============================] - 12s 328us/step - loss: 0.6379 - accuracy: 0.6234 - val_loss: 0.6378 - val_accuracy: 0.6242
Epoch 5/5
37924/37924 [==============================] - 12s 328us/step - loss: 0.6374 - accuracy: 0.6238 - val_loss: 0.6377 - val_accuracy: 0.6256


In [36]:
model.evaluate(X_test, y_test.to_pandas())

15170/15170 [==============================] - 4s 262us/step - loss: 0.6385 - accuracy: 0.6251


[0.6384584307670593, 0.6250947713851929]

## Predict

In [37]:
predict_like = model.predict(X_test)

In [38]:
predict_like

array([[0.5505934 , 0.4494066 ],
       [0.5172505 , 0.48274958],
       [0.5831792 , 0.41682082],
       ...,
       [0.37265196, 0.627348  ],
       [0.49968565, 0.5003143 ],
       [0.5505934 , 0.4494066 ]], dtype=float32)

In [39]:
predict_like = list(map(lambda x: 0 if x[0] > x[1] else 1,  predict_like))

In [40]:
y_test['predict_like'] = predict_like

In [41]:
rce_like = compute_rce(y_test['predict_like'].to_array(), y_test['is_like'].to_array())
rce_like

-1826.5365109006596

In [42]:
average_precision_score(y_test['predict_like'].to_array(), y_test['is_like'].to_array())

0.29836524175170515